### 구조화된 출력
- JSON 스키마를 준수하는 응답을 보장
- 응답 형식
- 함수 호출

In [1]:
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

True

In [2]:
client = OpenAI()

# json 모델이 지원되는 버전은 정해져 있음
Model = "gpt-4o-mini-2024-07-18"

#### pydantic.BaseModel
- python 의 데이터 검증 및 설정 관리 라이브러리
- 데이터 유효성 검증 및 JSON 직렬화(Serialization) 가능

In [3]:
# 응답형식 클래스 작성
from pydantic import BaseModel

class CalendarEvent(BaseModel):
    name : str
    date : str
    participants : list[str]

In [5]:
# create 와 달리 parse 를 사용하면 text_format 지정 가능

response = client.responses.parse(
    model=Model,
    input=[
        {"role":"developer", "content":"이벤트 정보를 추출하세요."},
        {"role":"user", "content":"Alice 와 Bob 은 금요일에 과학 박람회에 갈 예정입니다."},
    ],
    text_format=CalendarEvent
)

In [7]:
print(response.output_text) # 정해진 형식 => 문자열
print(response.output_parsed) # 정해진 형식

{"name":"과학 박람회","date":"금요일","participants":["Alice","Bob"]}
name='과학 박람회' date='금요일' participants=['Alice', 'Bob']


#### COT(Chain of Thought)
- 대형 언어모델에게 복잡한 문제를 풀 때, 답만 바로 내놓지 말고 그 과정을 단계별로 표현하도록 유도
- 모델이 답변을 단계별로 구조화하여 출력하도록 요청 가능

In [9]:
class Step(BaseModel):
    explanation : str
    output : str

class MathReasoning(BaseModel):
    steps : list[Step]
    final_answer : str

response = client.responses.parse(
    model=Model,
    input=[
        {"role":"developer", "content":"당신은 유용한 수학 튜터입니다. 사용자가 해결과정을 단계별로 따라갈 수 있도록 안내하세요."},
        {"role":"user", "content":"8x + 7 = -23 방정식 문제의 해는?"},
    ],
    text_format=MathReasoning,
    temperature=0
)

In [10]:
print(response.output_text)
print(response.output_parsed)

{"steps":[{"explanation":"주어진 방정식은 8x + 7 = -23입니다. 먼저, 양변에서 7을 빼줍니다.","output":"8x + 7 - 7 = -23 - 7"},{"explanation":"이제 방정식은 8x = -30이 되었습니다. 다음으로, 양변을 8로 나눕니다.","output":"8x / 8 = -30 / 8"},{"explanation":"따라서 x = -30 / 8입니다. 이를 간단히 하면 x = -15 / 4가 됩니다.","output":"x = -15 / 4"}],"final_answer":"x = -3.75 또는 x = -15/4"}
steps=[Step(explanation='주어진 방정식은 8x + 7 = -23입니다. 먼저, 양변에서 7을 빼줍니다.', output='8x + 7 - 7 = -23 - 7'), Step(explanation='이제 방정식은 8x = -30이 되었습니다. 다음으로, 양변을 8로 나눕니다.', output='8x / 8 = -30 / 8'), Step(explanation='따라서 x = -30 / 8입니다. 이를 간단히 하면 x = -15 / 4가 됩니다.', output='x = -15 / 4')] final_answer='x = -3.75 또는 x = -15/4'


In [11]:
response.output_parsed.model_dump()

{'steps': [{'explanation': '주어진 방정식은 8x + 7 = -23입니다. 먼저, 양변에서 7을 빼줍니다.',
   'output': '8x + 7 - 7 = -23 - 7'},
  {'explanation': '이제 방정식은 8x = -30이 되었습니다. 다음으로, 양변을 8로 나눕니다.',
   'output': '8x / 8 = -30 / 8'},
  {'explanation': '따라서 x = -30 / 8입니다. 이를 간단히 하면 x = -15 / 4가 됩니다.',
   'output': 'x = -15 / 4'}],
 'final_answer': 'x = -3.75 또는 x = -15/4'}

In [12]:
from PyPDF2 import PdfReader

def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)

    total_text = ""
    for page in reader.pages:
        total_text += page.extract_text()

In [13]:
total_text = extract_text_from_pdf("./data/deep_seek.pdf")

class ResearchPaperExtraction(BaseModel):
    title : str
    authors : list[str]
    abstract : str
    keywords : list[str]

response = client.responses.parse(
    model=Model,
    input=[
        {"role":"developer", "content":"당신은 구조화된 데이터 추출 전문가입니다. "
         "연구 논문의 비정형 텍스트를 제공받으면 이를 주어진 구조로 변환해야 합니다. 변환할 때 한국어로 번역해 주세요."},
        {"role":"user", "content":f"{total_text}"},
    ],
    text_format=ResearchPaperExtraction
)

In [14]:
print(response.output_parsed)
print(response.output_parsed.model_dump())

title='ResearchPaperExtraction' authors=[] abstract='' keywords=[]
{'title': 'ResearchPaperExtraction', 'authors': [], 'abstract': '', 'keywords': []}


In [22]:
from enum import Enum
from typing import Optional


class Category(str, Enum):
    폭력 = "violence"
    성적인_내용 = "sexual"
    자해 = "self_harm"


class ContentCompliance(BaseModel):
    is_violating: bool
    category: Optional[Category]
    explanation_if_violating: Optional[str]


response = client.responses.parse(
    model=Model,
    input=[
        {
            "role": "developer",
            "content": "사용자 입력이 특정 지침을 위반하는지 판단하고, 위반하는 경우 그 이유를 한국어로 설명하세요",
        },
        # {"role": "user", "content": "요즘 너무 힘들고 삶을 포기하고 싶어."},
        # {"role": "user", "content": "이 사진은 성적으로 노골적인 내용을 포함하고 있어"},
        {"role": "user", "content": "취업 면접을 준비하는 방법은 무엇인가요?"},
    ],
    text_format=ContentCompliance,
)

In [23]:
print(response.output_parsed.model_dump())

{'is_violating': False, 'category': None, 'explanation_if_violating': None}
